In [10]:
#some imports
from bs4 import BeautifulSoup
from dateutil import parser
from difflib import SequenceMatcher
import re,os
import glob

In [ ]:
#FOR TOPIC MODELLING
#https://de.dariah.eu/tatom/topic_model_python.html
import numpy as np  # a conventional alias

import sklearn.feature_extraction.text as text
from sklearn import decomposition

In [31]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [12]:
def language_detection_with_pyenchant(string_to_read):
    #https://stackoverflow.com/questions/3788870/how-to-check-if-a-word-is-an-english-word-with-python
    import enchant
    us = enchant.Dict("en_US")
    fr = enchant.Dict("fr_FR")
    lg_fr=0
    lg_ang=0
    for word in string_to_read:
        if fr.check(word) == True:
            lg_fr+=1
        if us.check(word) == True:
            lg_ang+=1
        
    if lg_fr >= lg_ang :
        return "french"
    else:
        return "english"
    

In [ ]:
def topic_analysis_with_mallet(string_to_read):
    lng = language_detection_with_pyenchant(string_to_read)
    vectorizer = text.CountVectorizer(input=string_to_read, stop_words=lng, min_df=20)
    dtm = vectorizer.fit_transform(filenames).toarray()
    vocab = np.array(vectorizer.get_feature_names())
    dtm.shape
    len(vocab)
    num_topics = 20
    num_top_words = 20
    clf = decomposition.NMF(n_components=num_topics, random_state=1)
    doctopic = clf.fit_transform(dtm)
    topic_words = []
    for topic in clf.components_:
        word_idx = np.argsort(topic)[::-1][0:num_top_words]
        topic_words.append([vocab[i] for i in word_idx])
    doctopic = doctopic / np.sum(doctopic, axis=1, keepdims=True)
    return [topic_words,doctopic]

In [13]:
def read_html_pages_in_dirs_and_extract_mails_dict(pages_dir):
    dict_mails={}
    #rootdir="./DHFR_sample/"
    count_Cc=0
    count_message=0
    for subdir, dirs, files in os.walk(pages_dir):
#ON VA LIRE TOUS LES FICHIERS DU REP ROOTDIR
        
        for file in files:
    #V1     if file.endswith('.html') == True  and file != ("index.html") and not re.match("mail\d+.html",file) : 
           if re.match("msg\d+.html",file) : 
          
            # les index.html sont des recap, on pourrait les compter pour s'assurer du nombre de messages
                
                #!print "\n",os.path.join(subdir, file), "\n"
                champs_mail={}
                filename=os.path.join(subdir, file)
                soup=BeautifulSoup(open(filename, 'r').read(),'lxml')#, 'html.parser') 
#RECUPERER LE SUJET DU MESSAGE
                
                i=0
                for zone_cible in soup.findAll('ul'):
                    i+=1
                    sujet_messg = zone_cible.find(string=re.compile("\[DH\]"))
                    zone_de_metadonnees = None
                    special_auteur=0
##ON ISOLE LE CHAMPS AUTEUR
                    if '<li><strong>From</strong>:' in str(zone_cible) and '<li><strong>To</strong>:' in str(zone_cible) :
                        #print "FROM/TO DETECTED, i= ",i
                        count_message+=1
                        #print  zone_cible
                        nom_auteur= re.sub(r'<ul>\n<li><strong>From</strong>: ', '',str(zone_cible))
                        sep = "&lt;"
                        nom_auteur = nom_auteur.split(sep, 1)[0]
                        
                        
                        mail1= re.sub(r'<ul>\n<li><strong>From</strong>:.+\n.+\n.+\n document\.write\(\"', '',str(zone_cible))
                        mail2=re.sub(r'<ul>\n<li><strong>From</strong>:.+\n.+\n.+\n document\.write\(\".+@\" \+ \"', '',str(zone_cible))
                        sep1 = "\" + \"@\" + "
                        sep2 = "\")"
                        mail1= mail1.split(sep1, 1)[0]
                        mail2= mail2.split(sep2, 1)[0]
                        mail_auteur= mail1+"@"+mail2
                        #print mail_auteur
                        if len(mail_auteur)>60:
                            try:
                                mail= re.sub(r'<ul>\n<li><strong>From</strong>:', '',str(zone_cible))
                                mail=re.search('&lt;(.*)&gt;',str(mail))
                                mail=mail.group(1)
                                #print "MAILBIS",mail
                                mail_auteur=mail
                                #special_auteur=1
                            except:
                                print "BUG"
                        #CHECK PROPER CAPTURE OF THE NAME    
                        if len(nom_auteur)>50:
                            #print zone_cible
                            nom_auteur=mail_auteur
                        if len(nom_auteur)<5:
                            nom_auteur=mail_auteur
                        if nom_auteur[0]== "\"":
                            nom_auteur=nom_auteur[1:]
                            nom_auteur=nom_auteur[:-2]
                            print nom_auteur
                        if nom_auteur.endswith(' '):
                            nom_auteur=nom_auteur[:-1]
                        #!print "nom_auteur ",nom_auteur+" mail_auteur "+mail_auteur,"\n"
                        champs_mail["nom_auteur"]=  nom_auteur
                        champs_mail["mail_auteur"]=  mail_auteur
                        champs_mail["ref_physique_de_l_article"]=  os.path.join(subdir, file)
                        champs_mail["sujet_du_message"] = sujet_messg
##ON ISOLE LE CHAMP DESTINATAIRES
                        #if special_auteur == 1:
                            #print "ZCspecAut",zone_cible
                        #else: 
                        
                        destinataires=re.sub(r'<ul>\n<li><strong>From</strong>:.+\n.+\n.+\n.+\n.+\n.+\n.+\n', '',str(zone_cible))
                        
                        sep3=";</li>"
                        destinataires= destinataires.split(sep3, 1)[0]
                        #print destinataires
                        dest=re.findall("<script type=.*\n.+\n.+\n.+\n.+/script>", destinataires)
                        #print dest
                        liste_destinataires=[]
                        for destinataire in dest:
                            #print destinataire,"\n___________________\n"
                            mail1= re.sub(r'<script type=\"text/javascript\">\n <!-- \n document\.write\(\"', '',str(destinataire))
                            mail2=re.sub(r'<script type=\"text/javascript\">\n <!-- \n document\.write\(\".+@\" \+ \"', '',str(destinataire))
                            sep1 = "\" + \"@\" + "
                            sep2 = "\")"
                            mail1= mail1.split(sep1, 1)[0]
                            mail2= mail2.split(sep2, 1)[0]
                            mail_destinataire= mail1+"@"+mail2
                            #print mail_destinataire,"\n___________\n"
                            if mail_destinataire not in liste_destinataires :
                                liste_destinataires.append(mail_destinataire)
                        if liste_destinataires==[]:
                            #print zone_cible
                            liste_destinataires=['dh@groupes.renater.fr']
                        #!print "destinataires: ",liste_destinataires,"\n"
                        champs_mail["liste_mails_dests"] = liste_destinataires
                       
                        if '<li><strong>Cc</strong>:' in str(zone_cible):
##ON ISOLE LE CHAMP CC
                            #print "CC DETECTED, i= ",i
                            #print zone_cible
                            count_Cc+=1
                            CCfield= re.search('<li><strong>Cc</strong>:(.*)<li><strong>Subject</strong>',str(zone_cible),flags=re.DOTALL)
                            CC= CCfield.group(1)
                            CC=str(CC)
                            CC=re.sub(r'^.+\n','',str(CC),flags=0)
                            CCs=re.findall("<script type=.*\n.+\n.+\n.+\n.+/script>", CC)
                            #sep1 = "\" + \"@\" + "
                            #mail1= mail1.split(sep1, 1)[0]
                            #print "CC FIELd\n"#,CCs
                            liste_CCs=[]
                            for CC in CCs:
                                #print CC,"\n___________________\n"
                                mail1= re.sub(r'<script type=\"text/javascript\">\n <!-- \n document\.write\(\"', '',str(CC))
                                mail2=re.sub(r'<script type=\"text/javascript\">\n <!-- \n document\.write\(\".+@\" \+ \"', '',str(CC))
                                sep1 = "\" + \"@\" + "
                                sep2 = "\")"
                                mail1= mail1.split(sep1, 1)[0]
                                mail2= mail2.split(sep2, 1)[0]
                                mail_CC= mail1+"@"+mail2
                                #print mail_CC,"\n___________\n"
                                if mail_CC not in liste_CCs :
                                    liste_CCs.append(mail_CC)
                            #!print "CCs: ",liste_CCs
                            champs_mail["liste_CCs"] = liste_CCs
##ON ISOLE LE CHAMP DATE                       
                        #print "ZONECIBLE",zone_cible
                        champs_datef=re.search('<li><strong>Date</strong>:(.+?)</li>',str(zone_cible),flags=re.DOTALL)
                        #print "CHAMPSDATEF",champs_datef
                        try :
                            champs_date=champs_datef.group(1)
                        except :
                            #print "ZCparent", zone_cible.parent
                            champs_date=re.search('<!--X-Date: (.*) -->',str(soup))
                            champs_date=champs_date.group(1)
                            #print champs_date
                            
                        #print champs_date
                        try : 
                            champs_date=parser.parse(champs_date)
                        except ValueError :
                            sep=" ("
                            champs_date=champs_date.split(sep, 1)[0]
                            
                        #!print  champs_date
                        champs_mail["date"] = champs_date
                        
                        
                        dict_mails[os.path.join(subdir, file)]= champs_mail
                        break
##LE DICTIONNAIRE DES MAILS SE NOMME dict_mails
    #print dict_mails
    print count_message," messages"
    print count_Cc, " messages avec Cc"
    ratiocount = float(count_Cc)/float(count_message)
    print "Taux_de_mails_avec_CC:",ratiocount,"\n"
    return dict_mails

In [583]:
def read_corpus_and_extract_topo_network(corpus):
    return

In [584]:
def read_corpus_and_extract_topo_Geo_network(corpus):
    return

In [14]:
def read_corpus_and_extract_mails(corpus):
    liste_mails=[]
    for mesg in corpus:
        #print corpus[mesg]
        if corpus[mesg]['mail_auteur'] not in liste_mails:
            liste_mails.append(corpus[mesg]['mail_auteur'])
        for i,key in enumerate(corpus[mesg]['liste_mails_dests']):
            #print key
            if key not in liste_mails:
                liste_mails.append(key)
        if 'liste_CCs' in corpus[mesg] :
            #print "TRUVE CC"
            for i,key in enumerate(corpus[mesg]['liste_CCs']):
                #print key
                if key not in liste_mails:
                    liste_mails.append(key)
    return liste_mails

In [520]:
def read_corpus_and_extract_localisations(corpus):
    return

In [15]:
def read_corpus_and_extract_names(corpus):
    liste_noms=[]
    for mesg in corpus:
        try: 
            if corpus[mesg]['nom_auteur'][0]== "\"":
                print corpus[mesg]['nom_auteur']
        except IndexError:
            print "Bug", corpus[mesg]['nom_auteur']
            print corpus[mesg]
        if [corpus[mesg]['nom_auteur'],corpus[mesg]['mail_auteur']] not in liste_noms:
            for i,key in enumerate(liste_noms):
                if SequenceMatcher(None, corpus[mesg]['nom_auteur'], key[0]).ratio() >=0.9:
                    print "POSSIBLE DUPLICATE |", corpus[mesg]['nom_auteur'],"|   |",key[0],"|"
                    print corpus[mesg]['mail_auteur'], key[1]
                
            liste_noms.append([corpus[mesg]['nom_auteur'],corpus[mesg]['mail_auteur']])
        
    return liste_noms

In [16]:
corpus_mails_DH=read_html_pages_in_dirs_and_extract_mails_dict("./DHFRsample")
print corpus_mails_DH



Noiret, Serge
Noiret, Serge
Ghislain SILLAUME
Noiret, Serge
aude.da-cruz-lima
Wandl-Vogt, Eveline
Wandl-Vogt, Eveline
elisabeth.belmas
Viera Rebolledo-Dhuin
Antonio A. Casilli
Anne Gresillon
Antonio A. Casilli
Epler, Jakob
Noiret, Serge
Manuel Zacklad
294  messages
35  messages avec Cc
Taux_de_mails_avec_CC: 0.119047619048 

{'./DHFRsample/2017-06/msg00011.html': {'nom_auteur': 'Jorge Fins', 'liste_mails_dests': ['dh@groupes.renater.fr'], 'mail_auteur': 'jorge.fins@univ-tours.fr', 'ref_physique_de_l_article': './DHFRsample/2017-06/msg00011.html', 'sujet_du_message': u': Re: [DH] Prestation de d\xe9veloppement sous drupal', 'date': datetime.datetime(2017, 6, 8, 13, 51, 2, tzinfo=tzlocal())}, './DHFRsample/2018-03/msg00011.html': {'nom_auteur': 'dumouchel suzanne', 'liste_mails_dests': ['legram@yahoogroupes.fr', 'culture.numerique@ml.free.fr', 'dh@groupes.renater.fr', 'listesocius@groups.openedition.org'], 'mail_auteur': 'dumouchelsuzanne@yahoo.fr', 'ref_physique_de_l_article': './DHFRsa

In [594]:
liste_mails=read_corpus_and_extract_mails(corpus_mails_DH)
print len(liste_mails)
print liste_mails


282
['jorge.fins@univ-tours.fr', 'dh@groupes.renater.fr', 'dumouchelsuzanne@yahoo.fr', 'legram@yahoogroupes.fr', 'culture.numerique@ml.free.fr', 'listesocius@groups.openedition.org', 'antoine.courtin@mac.com', 'ontologie-patrimoine@services.cnrs.fr', 'laetitia.bontemps@univ-tours.fr', 'DH@cru.fr', 'stephane.pouyllau@huma-num.fr', 'georges-xavier.blary@unilim.fr', 'Eveline.Wandl-Vogt@oeaw.ac.at', 'exploreAT@oeaw.ac.at', 'antonio.casilli@ehess.fr', 'frederic@clavert.net', 'antoine.blanchard@gmail.com', 'Nicole.Dufournaud@laposte.net', 'marjorie.burghart@ehess.fr', 'jean-daniel.fekete@inria.fr', 'pabloruizfabo@gmail.com', 'dh@cru.fr', 'oledeuff@gmail.com', 'pierre.mounier@openedition.org', 'claire.clivaz@unil.ch', 'christine.chadier@univ-lyon3.fr', 'aude.da-cruz-lima@mae.u-paris10.fr', 'archives-son-audiovisuel@groups.openedition.org', 'jerome.valluy@univ-paris1.fr', 'alexandre.hocquet@univ-lorraine.fr', 'annael.le-poullennec@psl.eu', 'elenagonzalezblanco@yahoo.es', 'tei-l@listserv.brown.

In [17]:
liste_noms=read_corpus_and_extract_names(corpus_mails_DH)
print len(liste_noms)
print liste_noms

POSSIBLE DUPLICATE | Pierre Mounier |   | Pierre Mounier |
pierre.mounier@openedition.org pierre.mounier@ehess.fr
POSSIBLE DUPLICATE | Clarisse Bardiot |   | Clarisse Bardiot |
clarisse_bardiot@me.com clarisse_bardiot@mac.com
POSSIBLE DUPLICATE | Claire Clivaz |   | Claire Clivaz |
claire.clivaz@unil.ch claire.clivaz@sib.swiss
POSSIBLE DUPLICATE | Stéphane Pouyllau |   | Stéphane Pouyllau |
stephane.pouyllau@cnrs.fr stephane.pouyllau@huma-num.fr
POSSIBLE DUPLICATE | Marin Dacos |   | Marin Dacos |
marin.dacos@openedition.org marin.dacos@revues.org
POSSIBLE DUPLICATE | Antonio Casilli |   | Antonio A. Casilli |
antonio.casilli@googlemail.com antonio.casilli@ehess.fr
POSSIBLE DUPLICATE | Sandrine Clérisse |   | Sandrine Clérisse |
sandrine.clerisse@cnrs.fr sclerisse@parisnanterre.fr
POSSIBLE DUPLICATE | Frédéric Clavert |   | Frédéric Clavert |
Frederic.Clavert@unil.ch frederic@clavert.net
POSSIBLE DUPLICATE | Jouni Tuominen |   | Jouni Tuominen |
jouni.tuominen@helsinki.fi jouni.tuomine

In [39]:
def read_html_pages_in_dirs_and_testtextextract(pages_dir):
    dict_mails={}
    #rootdir="./DHFR_sample/"
    count_Cc=0
    count_message=0
    for subdir, dirs, files in os.walk(pages_dir):
#ON VA LIRE TOUS LES FICHIERS DU REP ROOTDIR
        
        for file in files:
    #V1     if file.endswith('.html') == True  and file != ("index.html") and not re.match("mail\d+.html",file) : 
           if re.match("msg\d+.html",file) : 
            filename=os.path.join(subdir, file)
            with open(filename, 'r') as myfile:
                data=myfile.read()
                result = re.search('<!--X-Body-of-Message-->(.*)<!--X-Body-of-Message-End-->', data, flags=re.DOTALL)
                ZC= result.group(1)
                ZC=cleanhtml(ZC)
            
            
    return ZC

In [41]:
corpus_txt=read_html_pages_in_dirs_and_testtextextract("./DHFRsample")
print corpus_txt
lng= language_detection_with_pyenchant(corpus_txt)
print lng


Bonjour,&nbsp;Article dans la même veine. Nous sommes également intéressés par cette problématique pour faire gagner du temps aux internautes.&nbsp;https://www.academia.edu/12714248/Le_crowdsourcing_scientifique_et_patrimonial_%C3%A0_la_crois%C3%A9e_de_mod%C3%A8les_de_coordination_et_de_coop%C3%A9ration_h%C3%A9t%C3%A9rog%C3%A8nes_le_cas_des_herbiers_num%C3%A9ris%C3%A9s &nbsp;Bien cordialement, &nbsp;Manuel Zacklad&nbsp;Manuel ZACKLADDirecteur du laboratoire DICEN-IDF, Dispositifs d&#39;Information et de Communication à l&#39;Ère du Numérique en Ile de France - Dicen-IDF EA7339, CNAMProfesseur Titulaire de la Chaire Expressions et Cultures au Travail du CNAM - Département Culture, Information, Technique et Société (D6) - Ecole Management &amp; Société : 
 
 <!-- 
 document.write("manuel.zacklad" + "@" + "cnam.fr")
 // -->
 Adresse : CNAM – CITS - 16DP30 - 292, rue Saint Martin - 75141&nbsp;Paris Cedex 03Bureau : 2, rue de Conté, 75003 – M° Arts &amp; Métiers -&nbsp; 31.3.5A – Accès 31 

DictNotFoundError: Dictionary for language 'en_US' could not be found

In [47]:
import enchant
print enchant.list_languages()

[]
